**handson用資料としての注意点**

普通、同じセル上で何度も試行錯誤するので、最終的に上手くいったセルしか残らず、失敗したセルは残りませんし、わざわざ残しません。

今回はhandson用に 試行・思考過程を残したいと思い、エラーやミスが出ても下のセルに進んで処理を実行するようにしています。

notebookのセル単位の実行ができるからこそのやり方かもしれません。良い。

（下のセルから文は常体で書きます。）

kunai (@jdgthjdg)

---


In [ ]:
import sys;print(sys.version)

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
#設定でDataFrameなどが長く表示されないようにします（画面領域の消費を抑えてhandsonをしやすくするため）
# 長い場合の途中の省略表示(...)を出す閾値の設定（折り返しとは無関係）
pd.set_option('max_rows',10)
pd.set_option('max_columns',20) # これを超えたら全部は表示しない。 A B C ... X Y Z のように途中を省く。

In [ ]:
plt.rcParams["figure.figsize"] = [18, 6]
plt.rcParams['xtick.labelsize'] = 14 # 横軸のフォントサイズ
plt.rcParams['ytick.labelsize'] = 14 # 縦軸のフォントサイズ

In [ ]:
pwd

In [ ]:
ls ../../../data

In [ ]:
ls -lh ../../../data/2008_kansai/ #AMeDAS2008\(è╓É╝\).xls

## **花粉ライブラリ**
4. 「はなこさん」による過去データのダウンロード<BR> 
http://kafun.taiki.go.jp/library.html#4    

---

## エクセル(.xls/.xlsx) を読んでみる

pandas なら pd.read_XXX系 (csv, excel, sqlなど) を使えばほぼOK

非公式のサイトも良いですが、基本的に困ったら公式のドキュメントが良い。(結局こっちのが早くて確実だったり、知らないオプションを知る機会になる。自戒でもある。)  
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_excel.html
    
入出力系<BR>
https://pandas.pydata.org/pandas-docs/stable/reference/io.html

---

試しに化けまくりのパスをコピペして入れてみる

In [ ]:
df = pd.read_excel("../../../data/2008_kansai/ë╘ò▓âfü[â^2008(è╓É╝).xls")
df.head(3)

<br>

**読めた**

---
列名が **Unnamed: 1** とおかしい


**エクセルで開いたファイル** と見比べると、一行目はこのデータの名前(花粉自動測定器による観測データ)だけを載せているようだ  

なんとかならんかと公式ドキュメントを読む  
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_excel.html


>header : int, list of int, default 0  
>Row (0-indexed) to use for the column labels of the parsed DataFrame.  
>If a list of integers is passed those row positions will be combined into a MultiIndex. Use None if there is no header.
 
    とか  
    
>skiprows : list-like
>Rows to skip at the beginning (0-indexed).

skiprowsが楽そう

In [ ]:
df = pd.read_excel("../../../data/2008_kansai/ë╘ò▓âfü[â^2008(è╓É╝).xls", skiprows=1)
df.head()

ちゃんとしたヘッダーになった！

---

上の舞鶴のデータにある、 -9998 とかなんだ？？

**一番右の列(Unnamed: 25)に結構大事なことが書いてある。。。**

- -9998:降雪による欠測
- -9997:黄砂による欠測
- -9996:その他の欠測(前後の時間や周辺観測値と比較して不自然なデータ)
- 空白:未観測または通信障害による欠測


ちゃんと成形されてDBなどに保存されたデータなら、このような独自の列を使っての無理矢理のコメントはないが、  
担当者が手作業でぽちぽちと打った、ぬくもりのあるエクセルには良くある。


<br>
<br>
ぬくもりがあると、プログラムからはとても読みにくい・・・


もうメモったので右の2列は邪魔なので消す。  
邪魔なので消す。

# pandasでの列選択
---

 
**列だけなら　df \[ "列名" \]　が一番手っ取り早い。**

* 今回は列名というよりも右から2つという数値的な位置決めのが楽そう

### **xy座標っぽい選択なら　df.iloc[ 行(y軸), 列(x軸) ] を使う**

* 行は絞らないので、行部分は ' : '(全選択) で良い  
例: df.iloc[:, 3]  
例: df.iloc[3] # 二個目を省略すると行選択のみになる。列には ':' が入っているのと同等で全部が選択される。（pd.Series という型）

* 行も列もそれぞれ複数指定ができる  
例: df.iloc[:, [1,2] ]

* スライスも使える
例: df.iloc[2:4,:8]

In [ ]:
df.head() # 選択前のdf

In [ ]:
df.iloc[2] # 二個目を省略すると行選択

In [ ]:
df.iloc[1, 3]  # x,yを指定して、要素一つを抜き出す

In [ ]:
df.iloc[:3, [0,5] ]  # スライスやら、リストなどで複数指定

---

### 味のある右の2列を消す

In [ ]:
df.head(2)

In [ ]:
# スライスを使って左端から右の3列目までを選択
df = df.iloc[:,:-2]
df.head(3)

---

ちなみに・・・　2018年などは xlsx形式だが、 skiprows=1して、右2列が不要という形式は同じっぽい

In [ ]:
df = pd.read_excel("../../../data/2018_kansai/ë╘ò▓âfü[â^2018(è╓É╝).xlsx", )
df.head(3)

ここでも
- -9998:降雪による欠測
- -9997:黄砂による欠測
- -9996:その他の欠測(前後の時間や周辺観測値と比較して不自然なデータ)
- 空白:未観測または通信障害による欠測

***

## ついでに、AMeDASのデータも読んでみる

In [ ]:
ame_path = "../../../data/2008_kansai/AMeDAS2008(è╓É╝).xls"
amedas = pd.read_excel(ame_path)
amedas.head(3)

In [ ]:
amedas.columns

In [ ]:
amedas["彦根市役所"].plot(figsize=(20,4));

---

複数のシートがあるのでそれも見る

In [ ]:
amedas1 = pd.read_excel(ame_path, sheet_name=1)
amedas1.head(3)


In [ ]:
amedas2 = pd.read_excel(ame_path, sheet_name=2)
amedas2.head(3)


In [ ]:
amedas3 = pd.read_excel(ame_path, sheet_name=3)
amedas3.head(3)


In [ ]:
amedas4 = pd.read_excel(ame_path, sheet_name=4)
amedas4.head(3)


In [ ]:
amedas = pd.read_excel(ame_path, sheet_name=5)
amedas.head(3)
# IndexErrorが出てきたので、もう無い。　おわり